# <font color = 'blue' >  PROYECTO DE SIMULACIÓN MATEMÁTICA


**Integrantes:**
    
   <font color = 'green' >    Teresa Castillo
    
   <font color = 'green' >  Fernanda Pinedo
    
   <font color = 'green' >  Ana Paula Rangel 



___
### <Strong> I. Contexto </Strong>

> <p style='text-align: justify;'> Existen causas macroeconómicas las cuales afectan la oferta y la demanda de divisas en el mercado forex. El análisis fundamental se centra tanto en la situación macroeconómica de un país determinado, como en la situación económica a nivel mundial o de un grupo de estados particulares y sus respectivas relaciones.
 
 
> <p style='text-align: justify;'>Debemos tener presente que el mercado funciona como un todo, y por regla general los cambios producidos en la economía terminan repercutiendo tarde o temprano en la oferta y demanda tanto de divisas, commodities y demás activos financieros. Por dichas razones este proyecto se enfoca en hacer trading en el mercado de divisas usando como herramienta el calendario de eventos económicos con el objetivo de hacer un seguimiento de las noticias financieras más importantes, las que pueden afectar a las economías y crear movimientos de precios dentro de la divisa peso/dólar. 


___
### <Strong> II. Objetivo del proyecto </Strong>

Construir :
* Tabla descargando todos los precios de la divisa peso/dólar del 14 de abril de 2017 al 10 abril de 2019 con granularidad de 5 minutos.
* Tabla clasificando los eventos de un indicador del calendario económico en 4 escenarios* por fechas.
* Tabla clasificando cada escenario en 4 características*, cada evento con ventanas de 21 datos (10 datos antes de la fecha del evento y 10 datos después)
* Tabla de operaciones en donde se conozca tipo de operación que se realiza (compra, venta), unidades, precio de apertura, precio de cierre, margen utilizado y comentarios del porque se hizo la operación.
* Tabla de balance de la cuenta donde se muestra cuánto se ganó o perdió con las operaciones realizadas y conocer el saldo final de la cuenta (100,000 usd). 


##### Consumer Price Index ex Food & Energy (MoM) - United States
<p style='text-align: justify;'>Para este trabajo se eligió como indicador el Índice de Precios al Consumidor (IPC) Ex Food & Energy publicado por el Departamento de Estadísticas Laborales de los EE. UU. Es una medida de los movimientos de precios mediante la comparación entre los precios minoristas de una cesta de compras representativa de bienes y servicios. En términos generales, una lectura alta se considera positiva (o alcista) para el USD, mientras que una lectura baja se considera negativa (o bajista). Dicho indicador se encuentra en el calendario económico de la plataforma Fxstreet.

https://www.fxstreet.com/economic-calendar

In [116]:
# -- ------------------ Importar módulos a utilizar ------------------ -- #

import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
from datetime import datetime, date, time, timedelta
import calendar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [62]:
# -- ------------------ Parámetros para Data Frame ------------------ -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [63]:
# -- ------------------ Parámetros para OANDA ------------------ -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

A1_OA_Ak =  "6ff" + "6703d22fd972709992df543492afe" + "-f954a9914f9b39f72fba177a23eb243" + "b" #mi api


In [64]:
# -- ------------------ Estructuras de datos ------------------ -- #

# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, según fechas y granularidad
# -- -- Fecha, Apertura, Máximo, Mínimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operación (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razón con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrió operación: compra/venta",
# -- -- "Se cerro operación: Con pérdida de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion máximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operación: 1) TP o SL, 2) Regla con indicador


In [65]:
# -- ------------------ Inicializar API de OANDA ------------------ -- #

api = API(access_token=A1_OA_Ak)

In [66]:
# -- ------------------ Función de fechas para descargar precios ------------------ -- #

from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

In [67]:
# -- ------------------ Vector de fechas por periodos de 15 días ------------------ -- #

start_date = date(2017, 4, 14)
end_date = date(2019, 4, 10)
fechas = date_range(start_date, end_date, 15, 'days')

F1 = fechas[0].strftime('%Y-%m-%dT%H:%M:%S')
F2 = fechas[1].strftime('%Y-%m-%dT%H:%M:%S')

In [68]:
# -- ------------------ Descarga de los precios de OANDA ------------------ -- #


params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

___
### <Strong> DataFrame1 Precios peso-dólar </Strong>

In [69]:
# -- ------------------ Muestra de descarga de los precios de OANDA ------------------ -- #

lis = []
for i in range(len(A1_Hist['candles'])-1):
        lis.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

#Precios históricos del peso dólar

pd_hist = pd.DataFrame(lis)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
pd_hist.set_index('TimeStamp')

,Open,High,Low,Close
TimeStamp,,,,
2017-04-14 04:00:00,18.57634,18.57638,18.57450,18.57556
2017-04-14 04:05:00,18.57564,18.57619,18.57479,18.57558
2017-04-14 04:10:00,18.57560,18.57568,18.57540,18.57547
2017-04-14 04:15:00,18.57550,18.57550,18.57534,18.57538
2017-04-14 04:20:00,18.57536,18.57560,18.56932,18.57492
...,...,...,...,...
2017-04-28 20:30:00,18.82808,18.82991,18.82721,18.82988
2017-04-28 20:35:00,18.82985,18.83002,18.82970,18.82970
2017-04-28 20:40:00,18.82972,18.82973,18.82614,18.82672


___
### <Strong> Indicador: Consumer Price Index ex Food & Energy (MoM)</Strong>

In [70]:
# -- ------------------ Importar datos del indicador deseado ------------------ -- #

indicador = pd.read_csv('https://raw.githubusercontent.com/anapaurangel/SPF_anapaula/master/Consumer%20Price%20Index%20ex%20Food%20%26%20Energy%20(MoM).csv')
pd.set_option('display.max_rows', 25)
indicador.set_index('DateTime')

,Actual,Consensus,Previous
DateTime,,,
4/10/19 12:30,0.1,0.2,0.1
3/12/19 12:30,0.1,0.2,0.2
2/13/19 13:30,0.2,0.2,0.2
1/11/19 13:30,0.2,0.2,0.2
12/12/18 13:30,0.2,0.2,0.2
11/14/18 13:30,0.2,0.2,0.1
10/11/18 12:30,0.1,0.2,0.1
9/13/18 12:30,0.1,0.2,0.2
8/10/18 12:30,0.2,0.2,0.2


___
### <Strong> Escenarios del indicador </Strong>

$A$ = $Actual$ ≥ $Consensus$ ≥ $Previous$

$B$ = $Actual$ ≥ $Consensus$ ≤ $Previous$

$C$ = $Actual$ ≤ $Consensus$ ≥ $Previous$

$D$ = $Actual$ ≤ $Consensus$ ≤ $Previous$

Donde:

$Actual$ : Es el valor actual del indicador.

$Consensus$ : Es el valor del indicador que se pronóstica.

$Previous$ : Es el valor anterior del indicador.

In [71]:
# -- ------------------ Definición de escenarios ------------------ -- #

escenarios = []

for i in range(len(indicador['Actual'])):
    if indicador['Actual'][i]>= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario A
        escenarios.append('A')
    elif indicador['Actual'][i]>= indicador['Consensus'][i]<=indicador['Previous'][i]:#Escenario B
        escenarios.append('B')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]>=indicador['Previous'][i]: #Escenario C
        escenarios.append('C')
    elif indicador['Actual'][i]<= indicador['Consensus'][i]<=indicador['Previous'][i]: #Escenario D
        escenarios.append('D')

___
### <Strong> DataFrame 4 escenario del indicador por fecha </Strong>

In [72]:
# -- ------------------ Elaboración de DataFrame con datos del indicador y escenario ------------------ -- #

tabla=pd.DataFrame(columns=['DateTime','Actual','Consensus', 'Previous', 'Scene'])
tabla['DateTime'] = indicador['DateTime']
tabla['Actual'] = indicador['Actual']
tabla['Consensus'] = indicador['Consensus']
tabla['Previous'] = indicador['Previous']
tabla['Scene'] = escenarios
pd.set_option('display.max_rows', 25)
tabla.set_index('DateTime')

,Actual,Consensus,Previous,Scene
DateTime,,,,
4/10/19 12:30,0.1,0.2,0.1,C
3/12/19 12:30,0.1,0.2,0.2,C
2/13/19 13:30,0.2,0.2,0.2,A
1/11/19 13:30,0.2,0.2,0.2,A
12/12/18 13:30,0.2,0.2,0.2,A
11/14/18 13:30,0.2,0.2,0.1,A
10/11/18 12:30,0.1,0.2,0.1,C
9/13/18 12:30,0.1,0.2,0.2,C
8/10/18 12:30,0.2,0.2,0.2,A


___
### <Strong> Ventanas: 10 datos antes del indicador y 10 datos después </Strong>

In [73]:
serie = []
for i in range(0,25):
    serie.append(i+1)

In [74]:
# -- Función para descargar las ventanas de precios históricos [Open, High, Low, Close] -- #

def ventana(F1,F2):
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    lista = []

    for i in range(len(A1_Hist['candles'])-1):
            lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                          'Open': A1_Hist['candles'][i]['mid']['o'],
                          'High': A1_Hist['candles'][i]['mid']['h'],
                          'Low': A1_Hist['candles'][i]['mid']['l'],
                          'Close': A1_Hist['candles'][i]['mid']['c'],
                          'Serie': serie[i]})

    pd_hist = pd.DataFrame(lista)
    pd_hist = pd_hist[['TimeStamp', 'Serie', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    #a = pd_hist.set_index('TimeStamp')
    return(pd_hist)

___
### <Strong> Características de los escenarios </Strong>

$C1$ = $High_{max}$ - $Low_{min}$  -> Volatilidad

$C2$ = $Open_{t0}$ - $Close_{t10}$  -> Dirección

$C3$ = $Open_{t0}$ - $Low_{t10}$   -> Draw Down

$C4$ = $Open_{t0}$ - $High_{t10}$   -> Draw Up

Donde:

${max}$ = Máximo valor de la ventana de precios

${min}$ = Mínimo valor de la ventana de precios

$t_{0}$ = Tiempo de ocurrencia del indicador

$t_{10}$ = Último dato de la ventana de precios

In [75]:
# -- ------------------ Definición de las características de los escenarios  ------------------ -- #

def caract(v):
    o = v['Open'].astype(float)
    c = v['Close'].astype(float)
    l = v['Low'].astype(float)
    h = v['High'].astype(float)

    caract1= max(h)-min(l)
    caract2= o[10]-c[20]
    caract3= o[10]-l[20]
    caract4= o[10]-h[20]
    c1=round(caract1,3)
    c2=round(caract2,3)
    c3=round(caract3,3)
    c4=round(caract4,3)
    return(c1,c2,c3,c4)

In [76]:
# -- -------------- Función para saber qué paso antes, High max o Low min, de la ventana  ------------------ -- #


def c1(v):
    v1_o = v["Open"][10]
    v1_c = v["Close"][10]
    #v["High"].astype(float)
    #v["Low"].astype(float)
    v["Serie"].astype(float)
    h = v.loc[v['High'] == max(v['High'])]
    l = v.loc[v['Low'] == min(v['Low'])]
    hh = float(h['Serie'])
    ll = float(l["Serie"])
    vol = hh - ll
    return(vol)

In [77]:
# -- --------------------------------------Ventana indicador 1 ---------------------------------- #

v = ventana(F1 = "2019-04-10T11:40:00Z",F2 = "2019-04-10T13:30:00Z")
v1=caract(v)
v1_o = v["Open"][10]
v1_c = v["Close"][20]
vol1=c1(v)


In [78]:
# -- --------------------------------------Ventana indicador 2 ---------------------------------- #

v = ventana(F1 = "2019-03-12T11:40:00Z",F2 = "2019-03-12T13:30:00Z")
v2=caract(v)
v2_o = v["Open"][10]
v2_c = v["Close"][20]
vol2=c1(v)


In [79]:
# -- --------------------------------------Ventana indicador 3 ---------------------------------- #

v = ventana(F1 = "2019-02-13T12:40:00Z",F2 = "2019-02-13T14:30:00Z")
v3=caract(v)
v3_o = v["Open"][10]
v3_c = v["Close"][20]
vol3=c1(v)

In [80]:
# -- --------------------------------------Ventana indicador 4 ---------------------------------- #

v = ventana(F1 = "2019-01-11T12:40:00Z",F2 = "2019-01-11T14:30:00Z")
v4=caract(v)
v4_o = v["Open"][10]
v4_c = v["Close"][20]
vol4=c1(v)

In [81]:
# -- --------------------------------------Ventana indicador 5 ---------------------------------- #

v = ventana(F1 = "2018-12-12T12:40:00Z",F2 = "2018-12-12T14:30:00Z")
v5=caract(v)
v5_o = v["Open"][10]
v5_c = v["Close"][20]
vol5=c1(v)

In [82]:
# -- --------------------------------------Ventana indicador 6 ---------------------------------- #

v = ventana(F1 = "2018-11-14T12:40:00Z",F2 = "2018-11-14T14:30:00Z")
v6=caract(v)
v6_o = v["Open"][10]
v6_c = v["Close"][20]
vol6=c1(v)

In [83]:
# -- --------------------------------------Ventana indicador 7 ---------------------------------- #

v = ventana(F1 = "2018-10-11T11:40:00Z",F2 = "2018-10-11T13:30:00Z")
v7=caract(v)
v7_o = v["Open"][10]
v7_c = v["Close"][20]
vol7=c1(v)

In [84]:
# -- --------------------------------------Ventana indicador 8 ---------------------------------- #

v = ventana(F1 = "2018-09-13T11:40:00Z",F2 = "2018-09-13T13:30:00Z")
v8=caract(v)
v8_o = v["Open"][10]
v8_c = v["Close"][10]
vol8=c1(v)

In [85]:
# -- --------------------------------------Ventana indicador 9 ---------------------------------- #

v = ventana(F1 = "2018-08-10T11:40:00Z",F2 = "2018-08-10T13:30:00Z")
v9=caract(v)
v9_o = v["Open"][10]
v9_c = v["Close"][20]
vol9=c1(v)

In [86]:
# -- --------------------------------------Ventana indicador 10 ---------------------------------- #

v = ventana(F1 = "2018-07-12T11:40:00Z",F2 = "2018-07-12T13:30:00Z")
v10=caract(v)
v10_o = v["Open"][10]
v10_c = v["Close"][20]
vol10=c1(v)

In [87]:
# -- --------------------------------------Ventana indicador 11 ---------------------------------- #

v = ventana(F1 = "2018-06-12T11:40:00Z",F2 = "2018-06-12T13:30:00Z")
v11=caract(v)
v11_o = v["Open"][10]
v11_c = v["Close"][20]
vol11=c1(v)

In [88]:
# -- --------------------------------------Ventana indicador 12 ---------------------------------- #

v = ventana(F1 = "2018-05-10T11:40:00Z",F2 = "2018-05-10T13:30:00Z")
v12=caract(v)
v12_o = v["Open"][10]
v12_c = v["Close"][20]
vol12=c1(v)

In [89]:
# -- --------------------------------------Ventana indicador 13 ---------------------------------- #

v = ventana(F1 = "2018-04-11T11:40:00Z",F2 = "2018-04-11T13:30:00Z")
v13=caract(v)
v13_o = v["Open"][10]
v13_c = v["Close"][20]
vol13=c1(v)

In [90]:
# -- --------------------------------------Ventana indicador 14 ---------------------------------- #

v = ventana(F1 = "2018-03-13T11:40:00Z",F2 = "2018-03-13T13:30:00Z")
v14=caract(v)
v14_o = v["Open"][10]
v14_c = v["Close"][20]
vol14=c1(v)

In [91]:
# -- --------------------------------------Ventana indicador 15 ---------------------------------- #

v = ventana(F1 = "2018-02-14T12:40:00Z",F2 = "2018-02-14T14:30:00Z")
v15=caract(v)
v15_o = v["Open"][10]
v15_c = v["Close"][20]
vol15=c1(v)

In [92]:
# -- --------------------------------------Ventana indicador 16 ---------------------------------- #

v = ventana(F1 = "2018-01-12T12:40:00Z",F2 = "2018-01-12T14:30:00Z")
v16=caract(v)
v16_o = v["Open"][10]
v16_c = v["Close"][20]
vol16=c1(v)

In [93]:
# -- --------------------------------------Ventana indicador 17 ---------------------------------- #

v = ventana(F1 = "2017-12-13T12:40:00Z",F2 = "2017-12-13T14:30:00Z")
v17=caract(v)
v17_o = v["Open"][10]
v17_c = v["Close"][20]
vol17=c1(v)

In [94]:
# -- --------------------------------------Ventana indicador 18 ---------------------------------- #

v = ventana(F1 = "2017-11-15T11:40:00Z",F2 = "2017-11-15T13:30:00Z")
v18=caract(v)
v18_o = v["Open"][10]
v18_c = v["Close"][20]
vol18=c1(v)

In [95]:
# -- --------------------------------------Ventana indicador 19 ---------------------------------- #

v = ventana(F1 = "2017-10-13T11:40:00Z",F2 = "2017-10-13T13:30:00Z")
v19=caract(v)
v19_o = v["Open"][10]
v19_c = v["Close"][20]
vol19=c1(v)

In [96]:
# -- --------------------------------------Ventana indicador 20 ---------------------------------- #

v = ventana(F1 = "2017-09-14T11:40:00Z",F2 = "2017-09-14T13:30:00Z")
v20=caract(v)
v20_o = v["Open"][10]
v20_c = v["Close"][10]
vol20=c1(v)

In [97]:
# -- --------------------------------------Ventana indicador 21 ---------------------------------- #

v = ventana(F1 = "2017-08-11T11:40:00Z",F2 = "2017-08-11T13:30:00Z")
v21=caract(v)
v21_o = v["Open"][10]
v21_c = v["Close"][20]
vol21=c1(v)

In [98]:
# -- --------------------------------------Ventana indicador 22 ---------------------------------- #

v = ventana(F1 = "2017-07-14T11:40:00Z",F2 = "2017-07-14T13:30:00Z")
v22=caract(v)
v22_o = v["Open"][10]
v22_c = v["Close"][20]
vol22=c1(v)

In [99]:
# -- --------------------------------------Ventana indicador 23 ---------------------------------- #

v = ventana(F1 = "2017-06-14T11:40:00Z",F2 = "2017-06-14T13:30:00Z")
v23=caract(v)
v23_o = v["Open"][10]
v23_c = v["Close"][20]
vol23=c1(v)

In [100]:
# -- --------------------------------------Ventana indicador 24 ---------------------------------- #

v = ventana(F1 = "2017-05-12T11:40:00Z",F2 = "2017-05-12T13:30:00Z")
v24=caract(v)
v24_o = v["Open"][10]
v24_c = v["Close"][20]
vol24=c1(v)

In [101]:
# -- --------------------------------------Ventana indicador 25 ---------------------------------- #

v = ventana(F1 = "2017-04-14T11:40:00Z",F2 = "2017-04-14T13:30:00Z")
v25=caract(v)
v25_o = v["Open"][10]
v25_c = v["Close"][20]
vol25=c1(v)

In [119]:
volatilidad=[vol1,vol2,vol3,vol4,vol5,vol6,vol7,vol8,vol9,vol10,vol11,vol12,vol13,vol14,vol15,
            vol16,vol17,vol18,vol19,vol20,vol21,vol22,vol23,vol24,vol25]

# -- -------For para conocer que sucedio primero: el max high o el min low de la característica 1 ----------------- #
# -- -------Esto con el objetivo de saber que decisión tomar (compra o venta) ----------------- #
operación=[]
for i in range(len(volatilidad)):
    if volatilidad[i]<0:
        operación.append('Negativa')
    else:
        operación.append('Positiva')
        
        

___
### <Strong> DataFrame 5 Escenario-Características </Strong>

In [120]:
# -- ------------------ DataFrame de las características por escenario ------------------ -- #

indice = tabla['Scene']
columnas = ['Volatilidad','Dirección', 'Draw Down', 'Draw Up']
df = pd.DataFrame(index=indice, columns=columnas)

df.iloc[0] = v1
df.iloc[1] = v2
df.iloc[2] = v3
df.iloc[3] = v4
df.iloc[4] = v5
df.iloc[5] = v6
df.iloc[6] = v7
df.iloc[7] = v8
df.iloc[8] = v9
df.iloc[9] = v10
df.iloc[10] = v11
df.iloc[11] = v12
df.iloc[12] = v13
df.iloc[13] = v14
df.iloc[14] = v15
df.iloc[15] = v16
df.iloc[16] = v17
df.iloc[17] = v18
df.iloc[18] = v19
df.iloc[19] = v20
df.iloc[20] = v11
df.iloc[21] = v22
df.iloc[22] = v23
df.iloc[23] = v24
df.iloc[24] = v25
df['Tipo de Volatilidad']=operación
df

,Volatilidad,Dirección,Draw Down,Draw Up,Tipo de Volatilidad
Scene,,,,,
C,0.06,0.022,0.025,0.014,Negativa
C,0.141,0.063,0.074,0.062,Negativa
A,0.102,-0.046,-0.044,-0.065,Positiva
A,0.089,-0.037,-0.018,-0.039,Positiva
A,0.119,0.035,0.052,0.034,Negativa
A,0.128,0.037,0.054,0.025,Negativa
C,0.141,0.04,0.042,0.023,Negativa
C,0.151,0.122,0.136,0.117,Negativa
A,0.182,-0.149,-0.107,-0.161,Positiva


> <p style='text-align: justify;'>En el DataFrame anterior se muestran las característias dependiendo el escenario. Un tipo de volatilidad negativa significa que el max hight de esa ventana sucedio primero que el min low lo cual significa que el precio la divisa se fue a la baja y de lo contrario  con la volatilidad positiva es que el precio de la divisa aumento ya que primero fue el min low y despues el max hight.En la gran mayoría de los casos cuando se presenta una volatilidad negativa la dirección es positiva. Dicha relación se cumple dentro del análisis de este proyecto.

> <p style='text-align: justify;'> Si cualquiera de los draw es positivo significa que despues de que se dio la noticia del indicador la cotización de la divisa se fue a la baja de lo contrario la cotización de la divisa se fue a la alta.

In [104]:
# -- ------------------ Vector de decisiones ------------------ -- #
Decision=[]
for i in range(len(escenarios)):
    if escenarios[i]=='A':
        Decision.append('Venta')
    elif escenarios[i]=='B':
        Decision.append('S/O')
    elif escenarios[i]=='C':
        Decision.append('Venta')

In [105]:
# -- ------------------ Vector de compra de unidades de divisa ------------------ -- #
Unidades=[]
for i in range(len(escenarios)):
    if escenarios[i]=='A':
        Unidades.append(1000)
    elif escenarios[i]=='B':
        Unidades.append(0)
    elif escenarios[i]=='C':
        Unidades.append(5000)
        
   

In [106]:
# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

In [107]:
# -- ------------------ Creación de Folios para las operaciones ------------------ -- #

folio= []
for i in range(0,25):
    folio.append(1+i)
folio.reverse()

In [108]:
# -- ------------------ Vectores de los Precios de Apertura y Cierre (Open y Close) ------------------ -- #

P_O = [v1_o, v2_o, v3_o, v4_o, v5_o, v6_o, v7_o, v8_o, v9_o, v10_o, v11_o, v12_o, v13_o, v14_o, v15_o, v16_o,
       v17_o, v18_o, v19_o, v20_o, v21_o, v22_o, v23_o, v24_o, v25_o]

P_C = [v1_c, v2_c, v3_c, v4_c, v5_c, v6_c, v7_c, v8_c, v9_c, v10_c, v11_c, v12_c, v13_c, v14_c, v15_c, v16_c,
       v17_c, v18_c, v19_c, v20_c, v21_c, v22_c, v23_c, v24_c, v25_c]

In [109]:
# -- ------------------ Comentarios de las operaciones ------------------ -- #

C = ["Indicador de Dirección bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección posible bajista",
    "Indicadores no concluyentes",
    "Indicadores no concluyentes",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección posible bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección bajista",
    "Indicador de Dirección bajista",
    'Indicador de Dirección bajista']


In [110]:
P_O = [float(i) for i in P_O]
P_C = [float(i) for i in P_C]

### <Strong>DataFrame 2 Operaciones </Strong>

In [111]:
# -- ------------------ DataFrame de operaciones ------------------ -- #

indice = tabla['DateTime']
columnas = ['Folio', 'Operación', 'Unidades', 'Precio Apertura', 'Precio Cierre', 'Margen', 'Comentario']
df2_operaciones = pd.DataFrame(index=indice, columns=columnas)
df2_operaciones['Folio'] = folio
df2_operaciones['Precio Apertura'] = P_O
df2_operaciones['Precio Cierre'] = P_C
df2_operaciones['Comentario'] = C
df2_operaciones['Unidades'] = Unidades
df2_operaciones['Operación'] = Decision
margen = (df2_operaciones['Precio Apertura']*df2_operaciones['Unidades'])/par0_cap
df2_operaciones['Margen'] = margen
df2_operaciones.iloc[::-1]

,Folio,Operación,Unidades,Precio Apertura,Precio Cierre,Margen,Comentario
DateTime,,,,,,,
4/14/17 12:30,1,Venta,5000,18.54064,18.50358,0.927032,Indicador de Dirección bajista
5/12/17 12:30,2,Venta,5000,18.86794,18.76202,0.943397,Indicador de Dirección bajista
6/14/17 12:30,3,Venta,5000,18.02255,17.97358,0.901127,Indicador de Dirección bajista
7/14/17 12:30,4,Venta,5000,17.64254,17.54488,0.882127,Indicador de Dirección bajista
8/11/17 12:30,5,Venta,5000,17.95869,17.92514,0.897934,Indicador de Dirección bajista
9/14/17 12:30,6,Venta,1000,17.76226,17.80138,0.177623,Indicador de Dirección posible bajista
10/13/17 12:30,7,Venta,5000,18.97126,18.87416,0.948563,Indicador de Dirección bajista
11/15/17 13:30,8,Venta,1000,19.17100,19.13943,0.191710,Indicador de Dirección posible bajista
12/13/17 13:30,9,Venta,5000,19.22145,19.19510,0.961072,Indicador de Dirección bajista


> <p style='text-align: justify;'> Después de realizar el análisis de los escenarios con base en las características, se concluye que el 100% de las veces que el escenario es C; el precio de cotización de la divisa baja, por lo que se decide vender 5,000 unidades. Siguiendo con el mismo análisis para el escenario A, se muestra que el 64% de los eventos el precio de la divisa decrece y en 36% ocasiones aumenta, por lo tanto se decide vender 1,000 debido a la probabilidad de que suba. Esto es para que la pérdida no sea tan significativa. Finalmente para el escenario B, la probabilidad es 50/50 por lo que no se puede tener un criterio de compra/venta.

### <Strong>  DataFrame 3 Cuenta </Strong>
    

In [112]:
# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrió operación: compra/venta",
# -- --"Se cerró operacion: Con pérdida de: / Con ganancia de:"

In [113]:
# -- ------------------ DataFrame del Balance de la cuenta ------------------ -- #

indice = tabla['DateTime']
columnas = ['Capital', 'Flotante', 'Ganancia/Pérdida', 'Balance', 'Comentario']
df3_cuenta = pd.DataFrame(index=indice, columns=columnas)

df3_cuenta['Capital'] = df2_operaciones['Unidades']*df2_operaciones['Precio Apertura']
df3_cuenta['Flotante'] = par0_cap-df3_cuenta['Capital']

ganancia = df2_operaciones['Precio Apertura']-df2_operaciones['Precio Cierre']
df3_cuenta['Ganancia/Pérdida'] = ganancia*Unidades

df3_cuenta['Balance'] = df3_cuenta['Capital']+df3_cuenta['Ganancia/Pérdida']+df3_cuenta['Flotante']

# For para agregar el comentario de ganancia o pérdida:

resultado=[]
for i in range(len(df3_cuenta['Ganancia/Pérdida'])):
    if df3_cuenta['Ganancia/Pérdida'][i]>0:
        resultado.append("Ganancia")
    elif df3_cuenta['Ganancia/Pérdida'][i]<0:
        resultado.append("Pérdida")
    elif df3_cuenta['Ganancia/Pérdida'][i]==0:
        resultado.append("N/A")

df3_cuenta['Comentario'] = resultado
df3_cuenta.iloc[::-1]

,Capital,Flotante,Ganancia/Pérdida,Balance,Comentario
DateTime,,,,,
4/14/17 12:30,92703.20,7296.80,185.30,100185.30,Ganancia
5/12/17 12:30,94339.70,5660.30,529.60,100529.60,Ganancia
6/14/17 12:30,90112.75,9887.25,244.85,100244.85,Ganancia
7/14/17 12:30,88212.70,11787.30,488.30,100488.30,Ganancia
8/11/17 12:30,89793.45,10206.55,167.75,100167.75,Ganancia
9/14/17 12:30,17762.26,82237.74,-39.12,99960.88,Pérdida
10/13/17 12:30,94856.30,5143.70,485.50,100485.50,Ganancia
11/15/17 13:30,19171.00,80829.00,31.57,100031.57,Ganancia
12/13/17 13:30,96107.25,3892.75,131.75,100131.75,Ganancia


In [114]:
# -- ---------------- Frecuencia de veces que se obtuvo ganancia, pérdida o no se realizó operación ------------- -- #

## For's para contar las frecuencias:

numero_g=[]
for i in range(len(df3_cuenta['Ganancia/Pérdida'])):
    if df3_cuenta['Ganancia/Pérdida'][i]>0:
        numero_g.append(1)
    elif df3_cuenta['Ganancia/Pérdida'][i]<0:
        numero_g.append(0)
    elif df3_cuenta['Ganancia/Pérdida'][i]==0:
        numero_g.append(0)

numero_p=[]
for i in range(len(df3_cuenta['Ganancia/Pérdida'])):
    if df3_cuenta['Ganancia/Pérdida'][i]>0:
        numero_p.append(0)
    elif df3_cuenta['Ganancia/Pérdida'][i]<0:
        numero_p.append(1)
    elif df3_cuenta['Ganancia/Pérdida'][i]==0:
        numero_p.append(0)
        
numero_n=[]
for i in range(len(df3_cuenta['Ganancia/Pérdida'])):
    if df3_cuenta['Ganancia/Pérdida'][i]>0:
        numero_n.append(0)
    elif df3_cuenta['Ganancia/Pérdida'][i]<0:
        numero_n.append(0)
    elif df3_cuenta['Ganancia/Pérdida'][i]==0:
        numero_n.append(1)
        
numero_g = sum(numero_g)
numero_p = sum(numero_p)
numero_n = sum(numero_n)
total = len(df3_cuenta['Ganancia/Pérdida'])

In [115]:
# -- -------- Porcentaje de veces que se obtuvo ganancia, pérdida o no se realizó operación ---------------- -- #

n_gan = (numero_g/total)*100
n_perd = (numero_p/total)*100
n_nada = (numero_n/total)*100

gan = sum(df3_cuenta['Ganancia/Pérdida'])

print('El rendimiento fue de', (gan/par0_cap)*100,'%')
print('Del 100% de los eventos se tuvo ganancia en un', n_gan,'% de las ocasiones')
print('Del 100% de los eventos se tuvo pérdida en un', n_perd,'% de las ocasiones')
print('Del 100% de los eventos no se realizó operación en un', n_nada,'% de las ocasiones')

El rendimiento fue de 3.73938 %
Del 100% de los eventos se tuvo ganancia en un 76.0 % de las ocasiones
Del 100% de los eventos se tuvo pérdida en un 16.0 % de las ocasiones
Del 100% de los eventos no se realizó operación en un 8.0 % de las ocasiones


> <p style='text-align: justify;'>Finalmente después de haber realizado análisis fundamental sobre la divisa peso/dólar se obtuvieron los siguientes resultados: 

> <p style='text-align: justify;'>-El rendimiento fue de 3.74 %
> <p style='text-align: justify;'>-Del 100% de los eventos se tuvo ganancia en un 76.0 % de las ocasiones
> <p style='text-align: justify;'>-Del 100% de los eventos se tuvo pérdida en un 16.0 % de las ocasiones
> <p style='text-align: justify;'>-Del 100% de los eventos no se realizó operación en un 8.0 % de las ocasiones


> <p style='text-align: justify;'>Por último este análisis permite a los traders comprender y observar de una manera más global lo que está sucediendo en los mercados financieros. Representa una oportunidad para obtener beneficios tanto para inversionistas que operan a largo plazo como para aquellos que quieran aprovechar determinados momentos de alta volatilidad para ganar dinero a corto plazo
